In [1]:
import numpy as np
import pandas as pd  
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
News =pd.read_csv('Train.csv')
News.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
News.shape

(20800, 5)

In [4]:
News.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
### Drop Nan Values because it has too many missing values
News = News.dropna()

In [6]:
## Get the Independent Features

X=News.drop('label',axis=1)

In [7]:
## Get the Dependent features
y=News['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.17.0'

In [12]:
# Import necessary modules from TensorFlow and Keras for building and training an LSTM model

# Embedding layer converts positive integer indices into dense vectors of fixed size
from tensorflow.keras.layers import Embedding

# pad_sequences is used to ensure all sequences in a list have the same length by padding them with zeros
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sequential model is a linear stack of layers in Keras
from tensorflow.keras.models import Sequential

# one_hot is a function to encode text into a sequence of integers
from tensorflow.keras.preprocessing.text import one_hot

# LSTM (Long Short-Term Memory) is a type of recurrent neural network (RNN) layer that is useful for learning long-term dependencies
from tensorflow.keras.layers import LSTM

# Dense is a fully connected layer that is used for outputting results from the model
from tensorflow.keras.layers import Dense

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
# Create a copy of the original dataset `X` to ensure the original data remains unchanged.
# This is useful for data preprocessing steps where modifications are needed without altering the original data.
messages = X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
# Reset the index of the DataFrame `messages` and update it in place.
# This is useful to ensure the DataFrame has a clean, sequential index after any filtering or modifications.
# Setting `inplace=True` means the operation is performed on the DataFrame directly, rather than returning a new DataFrame.
messages.reset_index(inplace=True)

In [17]:
# Importing necessary modules from the NLTK (Natural Language Toolkit) library and regular expressions:
# nltk: A leading platform for building Python programs to work with human language data.
import nltk

# re: Provides support for regular expressions, allowing for complex string manipulation and pattern matching.
import re

# stopwords: Import the list of common stopwords in the English language from the NLTK corpus.
# Stopwords are commonly used words (such as 'the', 'is', 'in') that are often removed from text data to improve text analysis.
from nltk.corpus import stopwords

In [18]:
# Download the list of stopwords from the NLTK corpus.
# Stopwords are common words that are typically removed from text data during preprocessing to improve analysis.
# This step ensures that the stopwords list is available for use in text processing tasks.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JATIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
### Dataset Preprocessing

# Import the PorterStemmer for stemming words (reducing them to their root form)
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# Initialize an empty list to hold the cleaned and processed text data
corpus = []

# Loop through each message in the dataset
for i in range(0, len(messages)):
    # Remove non-alphabetical characters from the 'title' field and replace them with spaces
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    
    # Convert all characters in the review to lowercase
    review = review.lower()
    
    # Split the review into individual words
    review = review.split()
    
    # Stem each word in the review and remove stopwords
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    
    # Join the processed words back into a single string
    review = ' '.join(review)
    
    # Append the cleaned review to the corpus
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [22]:
onehot_repr

[[722, 4971, 4556, 217, 3624, 3004, 1687, 4823, 4331, 804],
 [4623, 1824, 4179, 4275, 2903, 2422, 2966],
 [4067, 1924, 3674, 2502],
 [4950, 1687, 1366, 1809, 2603, 2822],
 [63, 2903, 4096, 1746, 1649, 61, 2903, 2041, 206, 2154],
 [1567,
  869,
  2527,
  1846,
  182,
  93,
  2143,
  1498,
  4284,
  1637,
  169,
  4153,
  3788,
  2428,
  2966],
 [561, 1016, 1990, 722, 918, 260, 1764, 1064, 3688, 2665, 3326],
 [704, 1653, 2158, 1705, 2491, 2251, 93, 2151, 3688, 2665, 3326],
 [994, 1962, 3813, 2377, 3185, 2980, 2190, 1804, 93, 4334],
 [502, 3247, 2534, 1805, 3100, 2011, 433, 3566],
 [4617, 4508, 4102, 1822, 3516, 700, 3798, 2451, 3686, 939, 3681],
 [1809, 252, 3624, 2980, 93, 2491],
 [906, 3044, 928, 4820, 3387, 1305, 429, 4956, 4986],
 [4935, 4251, 1315, 224, 1305, 1993, 2821, 3688, 2665, 3326],
 [3607, 3824, 543, 3564, 363, 3688, 2665, 3326],
 [4536, 88, 4906, 2587, 4051, 2365, 1720, 2247, 2492, 496],
 [469, 1255, 1824],
 [195, 1951, 750, 4962, 93, 90, 2586, 2966],
 [4472, 4863, 4179, 47

### Embedding Representation

In [23]:
# Set the maximum length of a sentence to 20
sent_length = 20

# Pad the one-hot representations of documents to a fixed length (sent_length)
# using zeros at the beginning of each sequence (padding='pre')
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

# Print the padded sequences
print(embedded_docs)

[[   0    0    0 ... 4823 4331  804]
 [   0    0    0 ... 2903 2422 2966]
 [   0    0    0 ... 1924 3674 2502]
 ...
 [   0    0    0 ... 3688 2665 3326]
 [   0    0    0 ... 3521 2578 4548]
 [   0    0    0 ... 1191 2204 1368]]


In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  722,
       4971, 4556,  217, 3624, 3004, 1687, 4823, 4331,  804])

In [25]:
## Creating model

# Define the number of features to represent each word in the embedding space
embedding_vector_features = 40 

# Create a sequential neural network model
model = Sequential()

# Add an embedding layer to convert words into dense vectors
# - voc_size: number of unique words in the vocabulary
# - embedding_vector_features: number of features to represent each word
# - input_length: length of each input sequence (sentence)
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))

# Add a Long Short-Term Memory (LSTM) layer with 100 units
# to learn sequential dependencies in the data
model.add(LSTM(100))

# Add a dense layer with 1 unit and sigmoid activation function
# to output a probability value (binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss function and Adam optimizer
# and track accuracy as a metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [26]:
len(embedded_docs),y.shape

(18285, (18285,))

In [27]:
import numpy as np

# Convert the padded sequences (embedded_docs) into a NumPy array
# to prepare it for training the model
X_final = np.array(embedded_docs)

# Convert the target variable (y) into a NumPy array
# to prepare it for training the model
y_final = np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [30]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8201 - loss: 0.4234 - val_accuracy: 0.9180 - val_loss: 0.1934
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9418 - loss: 0.1510 - val_accuracy: 0.9213 - val_loss: 0.1900
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9685 - loss: 0.0928 - val_accuracy: 0.9201 - val_loss: 0.1983
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9790 - loss: 0.0646 - val_accuracy: 0.9175 - val_loss: 0.2237
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9861 - loss: 0.0478 - val_accuracy: 0.9133 - val_loss: 0.2480
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9908 - loss: 0.0364 - val_accuracy: 0.9114 - val_loss: 0.3222
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9941 - loss: 0.0239 - val_accuracy: 0.9173 - val_loss: 0.3852
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9955 - loss: 0.0184 - val_accu

### Adding Dropout

In [31]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [32]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [33]:
# Convert the predicted probabilities to binary class labels
# by thresholding at 0.5
y_pred = np.where(y_pred > 0.5, 1, 0)

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(y_test,y_pred)

array([[3143,  276],
       [ 239, 2377]], dtype=int64)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9146644573322287

In [37]:
from sklearn.metrics import classification_report

In [38]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      3382
           1       0.91      0.90      0.90      2653

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

